In [1]:
import sys
import os

_path_added = False

def add_project_root_to_path():
    """
    Check if the project root directory is in the Python path.
    If not, add it to sys.path and change the working directory to the project root.
    """
    global _path_added

    if _path_added:
        return
    
    root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

    if root_dir not in sys.path:
        sys.path.append(root_dir)
        print(f"Added {root_dir} to Python path.")
    
    if os.getcwd() != root_dir:
        os.chdir(root_dir)
        print(f"Changed working directory to {root_dir}.")
    _path_added = True


add_project_root_to_path()

Added /home/developing_nacho/fhdw/web_social_analytics/dj-gpt/insta_scraping to Python path.
Changed working directory to /home/developing_nacho/fhdw/web_social_analytics/dj-gpt/insta_scraping.


In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dorfterror_database.database_structure import Base, Post, Profile
from pathlib import Path

root_path = Path(".").resolve()
print(root_path)

engine = create_engine(f"sqlite:///{root_path}/dorfterror_database/dorfterror.db")
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

/home/developing_nacho/fhdw/web_social_analytics/dj-gpt/insta_scraping


In [3]:
total_posts = session.query(Post).count()

# Top 25% erfolgreichste Posts
top_posts = (
    session.query(Post)
    .order_by(Post.like_count.desc())
    #.limit( int(total_posts * 0.25) )
    .all()
)

print(top_posts)
print(f"Anzahl der Posts: {len(top_posts)}")

[<dorfterror_database.database_structure.Post object at 0x76305a8f21b0>, <dorfterror_database.database_structure.Post object at 0x76305a8f1ca0>, <dorfterror_database.database_structure.Post object at 0x76305a8f14c0>, <dorfterror_database.database_structure.Post object at 0x76305a8f2210>, <dorfterror_database.database_structure.Post object at 0x76305a8f2240>, <dorfterror_database.database_structure.Post object at 0x76305a8f22a0>, <dorfterror_database.database_structure.Post object at 0x76305a8f2300>, <dorfterror_database.database_structure.Post object at 0x76305a8f2330>, <dorfterror_database.database_structure.Post object at 0x76305a8f2360>, <dorfterror_database.database_structure.Post object at 0x76305a8f2390>, <dorfterror_database.database_structure.Post object at 0x76305a8f23f0>, <dorfterror_database.database_structure.Post object at 0x76305a8f2450>, <dorfterror_database.database_structure.Post object at 0x76305a8f2480>, <dorfterror_database.database_structure.Post object at 0x76305a

In [4]:
from openai import OpenAI

# Initialize LM Studio client
client = OpenAI(base_url="http://192.168.178.85:1200/v1", api_key="lm-studio")
MODEL = "google/gemma-3-12b"


In [5]:
def get_category_of_user_request(user_request):
    """
    Get the category of a user request using the LM Studio client.
    """
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "Du bist eine Expertin darin, Instagram-Posts anhand ihres Inhalts zu kategorisieren. Diese Kategorien kannst du verwenden: „Musik & Releases“ (für z.B. neue Musik, Song-Hintergründe, Musikvideos oder Studio-Sessions), „Live & Tour“ (für z.B. Tourdaten, Konzertankündigungen, Fotos und Videos nach Auftritten oder Backstage/Behind-the-scenes Inhalte), „Bandalltag“ (für z.B. Proberaum-Szenen, Mitglieder vorstellen oder Throwbacks und Jubiläumsposts), „Politisches Engagement“ (für z.B. politische Themen und gesellschaftskritische Beiträge), „Merchandise“ (für z.B. Verlinkungen zum Shop, Merchandise Ankündigungen oder Rabattaktionen). ",
            },
            {
                "role": "user",
                "content": f"Du bekommst nun eine Anfrage von einem Nutzer, der einen neuen Instagram-Post erstellen möchte. Deine Aufgabe ist es, die Anfrage in eine der oben genannten Kategorien einzuordnen. Antworte nur mit der Kategorie, ohne weitere Erklärungen oder Kommentare: {user_request}",
            },
        ],
        max_tokens=500,
        temperature=0.5,
    )
    
    return response.choices[0].message.content.strip()

In [6]:
def get_posts_from_category(category, top_percent=1.0):
    """
    Get posts from a specific category.
    Ensures at least 5 posts are returned for top 5%.
    """
    total_posts = session.query(Post).filter(Post.category == category).count()

    if top_percent == 0.05:
        limit = max(5, int(total_posts * top_percent))
    else:
        limit = int(total_posts * top_percent)

    posts = (session.query(Post)
             .filter(Post.category == category)
             .order_by(Post.like_count.desc())
             .limit(limit)
             .all())

    return posts

In [7]:
import random

def get_context_posts(top_5percent_posts, top_80percent_posts):
    """    Get a sample of posts for context, ensuring no overlap between top 5% and top 80% posts.
    Returns a list of 5 posts: 3 from top 5% and 2 from top 80%, ensuring no overlap.
    """
    # Ensure no overlap between top_5percent_posts and top_80percent_posts
    top_5_ids = {post.id for post in top_5percent_posts}
    top_80_filtered = [post for post in top_80percent_posts if post.id not in top_5_ids]

    # Sample without replacement
    sample_5 = random.sample(top_5percent_posts, min(3, len(top_5percent_posts)))
    sample_80 = random.sample(top_80_filtered, min(2, len(top_80_filtered)))

    # Combine into one list
    context_posts = sample_5 + sample_80

    # (Optional) Shuffle final list
    random.shuffle(context_posts)

    return context_posts

In [8]:
def generate_post(user_request, top_5percent_posts, top_80percent_posts):
    """
    Generate a post based on user request and context posts.
    """
    category = get_category_of_user_request(user_request)
    
    # Get context posts
    context_posts = get_context_posts(top_5percent_posts, top_80percent_posts)
    
    # Prepare context for the model
    context_text = [post.caption for post in context_posts]
    
    # Generate post content using the LM Studio client
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "Du bist eine Expertin darin, Instagram-Posts zu generieren. Verwende die bereitgestellten Kontext-Posts, um einen neuen Post zu erstellen.",
            },
            {
                "role": "user",
                "content": f"Hier sind einige Kontext-Posts:\n{context_text}\n\nErstelle einen neuen Post für die Kategorie '{category}' basierend auf der Anfrage: {user_request}",
            },
        ],
        max_tokens=500,
        temperature=0.5,
    )
    
    return response.choices[0].message.content.strip()

In [11]:

user_prompt = input("Bitte beschreibe deinen Instagram-Post, welchen du erstellen willst: ")

user_prompt_category = get_category_of_user_request(user_prompt)

print(f"Die Kategorie für den Post lautet: {user_prompt_category}")

top_5percent_posts = get_posts_from_category(user_prompt_category, top_percent=0.05)

top_80percent_posts = get_posts_from_category(user_prompt_category, top_percent=0.8)

generated_post = generate_post(user_prompt, top_5percent_posts, top_80percent_posts)

print(f"Generierter Post: {generated_post}")


Die Kategorie für den Post lautet: Musik & Releases
Generierter Post: Okay, hier ist ein Instagram-Postentwurf, der auf den bereitgestellten Kontexten basiert und deine Anfrage berücksichtigt:

---

🎉 **SAVE THE DATE!** 🎉

Wir haben riesige Neuigkeiten für euch! Am 03.10.25 droppen wir unsere brandneue Single "Drei Akkorde, trotzdem nice" – und das ist erst der Anfang! 🔥

Diese Single ist der erste Vorgeschmack auf unser kommendes Album, an dem wir mit viel Herzblut gearbeitet haben. Lasst euch überraschen, was noch kommt! 🤫

Bleibt dran für weitere Infos & Updates! Wir können es kaum erwarten, unsere neue Musik mit euch zu teilen! ❤️

#neuesingle #dreiakkordetrotztnice #albumankündigung #punkrock #newmusic #comingsoon #musik #release #vorfreude [DeinBandName] 🤘
---

**Warum dieser Post so funktioniert:**

*   **Aufmerksamkeitsstark:** Der Anfang mit "SAVE THE DATE!" und den Emojis fängt die Aufmerksamkeit ein.
*   **Klar & Prägnant:** Die wichtigsten Informationen (Datum, Songtitel, A